In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 97433, done.
remote: Counting objects: 100% (713/713), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 97433 (delta 434), reused 591 (delta 375), pack-reused 96720
Receiving objects: 100% (97433/97433), 613.67 MiB | 27.54 MiB/s, done.
Resolving deltas: 100% (70846/70846), done.


In [3]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

In [4]:
!cd models/research/ && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
import os
import pathlib
import imageio
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image


In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu)

In [8]:
!wget -nc https://lazyprogrammer.me/course_files/cnn_class2_videos.zip

--2024-06-27 14:49:01--  https://lazyprogrammer.me/course_files/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  8.06MB/s    in 0.2s    

2024-06-27 14:49:02 (8.06 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [9]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             


In [10]:
INPUT_VIDEOS = ['catdog', 'safari', 'traffic']

In [11]:
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz"

In [13]:
PATH_TO_MODEL = tf.keras.utils.get_file(
    fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin=url,
    untar=True
)
PATH_TO_MODEL

386527459/386527459 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

In [16]:
# url = "https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt"
# PATH_TO_LABELS = tf.keras.utils.get_file(
#     fname="mscoco_label_map.pbtxt",
#     origin=url,
#     untar = False
# )
PATH_TO_LABELS = 'mscoco_label_map.pbtxt'

In [17]:
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


In [18]:
SAVED_MODEL_PATH = PATH_TO_MODEL + '/saved_model'

In [19]:
detect_fn = tf.saved_model.load(SAVED_MODEL_PATH)

In [20]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

In [49]:
def detect_objects_in_img(img_np):

  img_tensor = tf.convert_to_tensor(img_np)

  img_tensor = img_tensor[tf.newaxis, ...]

  detections = detect_fn(img_tensor)

  num_detections = int(detections.pop('num_detections'))

  detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}

  detections['num_detection'] = num_detections

  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  img_with_detections = img_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
      img_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.3,
      agnostic_mode=False
  )

  return img_with_detections

In [50]:
def detect_objects_in_videos(input_video):

  print(f'Running inference for {input_video}... ')

  video_reader = imageio.get_reader(f'{input_video}.mp4')
  video_writer = imageio.get_writer(f'{input_video}_annotated.mp4', fps=10)

  t0 = time.time()
  n_frames = 0
  for frame in video_reader:
    n_frames += 1
    new_frame = detect_objects_in_img(frame)

    video_writer.append_data(new_frame)

  fps = n_frames / (time.time() - t0)

  print(f'Frames Processed: {n_frames}, At Speed: {fps}')

  video_writer.close()

In [51]:
detect_objects_in_videos(INPUT_VIDEOS[0])

Running inference for catdog... 
Frames Processed: 50, At Speed: 0.20599391639836692


In [52]:
detect_objects_in_videos(INPUT_VIDEOS[1])

Running inference for safari... 


Frames Processed: 100, At Speed: 0.2095588581947261


In [53]:
detect_objects_in_videos(INPUT_VIDEOS[2])

Running inference for traffic... 
Frames Processed: 70, At Speed: 0.21582673862514393
